Check how long it takes to integrate each step of 12/41.3 vs 12.41.4

## Check 12/41.1

In [55]:
import pickle
import os
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTStateV2
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
# cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import argparse
import copy
from perses.dispersed import feptasks
import mdtraj as md
import numpy as np
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from openmmtools.integrators import LangevinIntegrator
import time
from tqdm import tqdm

In [56]:
# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)


In [72]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/12/41/"
resid = '29'
old_aa_name = 'ALA'
new_aa_name = 'TYR'
sim_number = 3
length = 1
move_length = 3
timestep = 4
radius = 0.2

if sim_number == 1:
    phase = 'apo'
    name = old_aa_name
    state = 0
elif sim_number == 2:
    phase = 'apo'
    name = new_aa_name
    state = 1
elif sim_number == 3:
    phase = 'complex'
    name = old_aa_name
    state = 0
elif sim_number == 4:
    phase = 'complex'
    name = new_aa_name
    state = 1



In [73]:
i = os.path.basename(os.path.dirname(outdir))
path = os.path.join(outdir, f"{i}_{phase}_{state}.pickle")
htf = pickle.load(open(path, "rb" ))


In [74]:
# Build REST factory
_logger.info("Generating REST factory")
_logger.info(f"radius:{radius} nm")
query_indices = [atom.index for atom in list(htf.hybrid_topology.residues)[int(resid)].atoms]
_logger.info(f"query indices {query_indices}")
traj = md.Trajectory(np.array(htf.hybrid_positions), htf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=solute_atoms)[0])
factory = RESTTopologyFactory(htf.hybrid_system, solute_region=rest_atoms)

_logger.info("Generating REST states")
# Get REST system
REST_system = factory.REST_system

# Create states for each replica
n_replicas = 12  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 1200 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

# Create reference thermodynamic state
lambda_zero_alchemical_state = RESTStateV2.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

context_cache = cache.ContextCache()

# Create thermodynamics states
sampler_state =  SamplerState(htf.hybrid_positions, box_vectors=htf.hybrid_system.getDefaultPeriodicBoxVectors())
beta_0 = 1/(kB*T_min)
thermodynamic_state_list = []
sampler_state_list = []
for temperature in temperatures:
    beta_m = 1/(kB*temperature)
    compound_thermodynamic_state_copy = copy.deepcopy(compound_thermodynamic_state)
    compound_thermodynamic_state_copy.set_alchemical_parameters(beta_0, beta_m)
    thermodynamic_state_list.append(compound_thermodynamic_state_copy)

    # now generating a sampler_state for each thermodynamic state, with relaxed positions
    # context, context_integrator = context_cache.get_context(compound_thermodynamic_state_copy)
    feptasks.minimize(compound_thermodynamic_state_copy, sampler_state)
    sampler_state_list.append(copy.deepcopy(sampler_state))

INFO:root:Generating REST factory
INFO:root:radius:0.2 nm
INFO:root:query indices [466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 29466, 29467, 29468, 29469, 29470, 29471, 29472, 29473, 29474, 29475, 29476, 29477, 29478, 29479, 29480]
INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=6.7997, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=6.7997, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=6.7997), unit=nanometer)]
INFO:REST:No unknown forces.
INFO:root:Generating REST states


In [75]:
# Define simulation parameters
temperature = 298 * unit.kelvin
nsteps = 250 # 1 ps
timestep = 4.0 * unit.femtosecond
platform_name = 'CUDA'
collision_rate = 1.0 / unit.picoseconds

In [76]:
# Set up integrator
integrator = LangevinIntegrator(temperature, collision_rate, timestep)

# Set up context
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')

context = thermodynamic_state_list[1].create_context(integrator, platform=platform)    

context.setPeriodicBoxVectors(*htf.hybrid_system.getDefaultPeriodicBoxVectors())
context.setPositions(htf.hybrid_positions)
context.setVelocitiesToTemperature(temperature)
print(compound_thermodynamic_state.reduced_potential(context))

# Minimize
openmm.LocalEnergyMinimizer.minimize(context)



-21509.12042669337


In [77]:
# Run equilibration
times = []
for step in tqdm(range(nsteps)):
    initial_time = time.time()
    integrator.step(1)
    elapsed_time = (time.time() - initial_time) * unit.seconds
    print(f'took: {elapsed_time / unit.seconds} seconds')
    times.append(elapsed_time / unit.seconds)


 55%|█████▌    | 138/250 [00:00<00:00, 664.05it/s]

took: 0.0024857521057128906 seconds
took: 0.0008969306945800781 seconds
took: 0.001018524169921875 seconds
took: 0.0011260509490966797 seconds
took: 0.001283884048461914 seconds
took: 0.0011010169982910156 seconds
took: 0.0010342597961425781 seconds
took: 0.0010597705841064453 seconds
took: 0.0011055469512939453 seconds
took: 0.0011339187622070312 seconds
took: 0.0010721683502197266 seconds
took: 0.00133514404296875 seconds
took: 0.0010554790496826172 seconds
took: 0.0010929107666015625 seconds
took: 0.0010938644409179688 seconds
took: 0.0011377334594726562 seconds
took: 0.0013585090637207031 seconds
took: 0.0011093616485595703 seconds
took: 0.0010852813720703125 seconds
took: 0.0010881423950195312 seconds
took: 0.0010924339294433594 seconds
took: 0.0010859966278076172 seconds
took: 0.0013577938079833984 seconds
took: 0.0011067390441894531 seconds
took: 0.0011060237884521484 seconds
took: 0.001088857650756836 seconds
took: 0.0011394023895263672 seconds
took: 0.0011010169982910156 secon

100%|██████████| 250/250 [00:00<00:00, 705.50it/s]


took: 0.0005240440368652344 seconds
took: 0.0012173652648925781 seconds
took: 0.0010132789611816406 seconds
took: 0.0010478496551513672 seconds
took: 0.006013631820678711 seconds
took: 0.0014224052429199219 seconds
took: 0.0010001659393310547 seconds
took: 0.0009989738464355469 seconds
took: 0.0010607242584228516 seconds
took: 0.0012171268463134766 seconds
took: 0.0036385059356689453 seconds
took: 0.0009922981262207031 seconds
took: 0.0010449886322021484 seconds
took: 0.0009927749633789062 seconds
took: 0.0010464191436767578 seconds
took: 0.0012230873107910156 seconds
took: 0.001005411148071289 seconds
took: 0.0010609626770019531 seconds
took: 0.001001119613647461 seconds
took: 0.0010030269622802734 seconds
took: 0.0012586116790771484 seconds
took: 0.0010361671447753906 seconds
took: 0.001018524169921875 seconds
took: 0.0010523796081542969 seconds
took: 0.0010404586791992188 seconds
took: 0.0012600421905517578 seconds
took: 0.001007080078125 seconds
took: 0.0010421276092529297 seconds

In [78]:
print(f"average {np.mean(times)}")

average 0.0012209577560424805


In [80]:
for i in range(2, 11):
    # Set up integrator
    integrator = LangevinIntegrator(temperature, collision_rate, timestep)

    # Set up context
    platform = openmm.Platform.getPlatformByName(platform_name)
    if platform_name in ['CUDA', 'OpenCL']:
        platform.setPropertyDefaultValue('Precision', 'mixed')
    if platform_name in ['CUDA']:
        platform.setPropertyDefaultValue('DeterministicForces', 'true')

    context = thermodynamic_state_list[i].create_context(integrator, platform=platform)    

    context.setPeriodicBoxVectors(*htf.hybrid_system.getDefaultPeriodicBoxVectors())
    context.setPositions(htf.hybrid_positions)
    context.setVelocitiesToTemperature(temperature)
    print(compound_thermodynamic_state.reduced_potential(context))

    # Minimize
    openmm.LocalEnergyMinimizer.minimize(context)

    # Run equilibration
    times = []
    for step in tqdm(range(nsteps)):
        initial_time = time.time()
        integrator.step(1)
        elapsed_time = (time.time() - initial_time) * unit.seconds
#         print(f'took: {elapsed_time / unit.seconds} seconds')
        times.append(elapsed_time / unit.seconds)
        
    print(f"state: {i}, average {np.mean(times)}")


-21548.11366847344


100%|██████████| 250/250 [00:00<00:00, 718.26it/s]


state: 2, average 0.0013539304733276368
-21580.469188256615


100%|██████████| 250/250 [00:00<00:00, 706.18it/s]


state: 3, average 0.0013791847229003907
-21607.68131340663


...

state: 8, average 0.0012280778884887695
-21696.084169535865


100%|██████████| 250/250 [00:00<00:00, 710.99it/s]


state: 9, average 0.0013694295883178711
-21707.696424263708


100%|██████████| 250/250 [00:00<00:00, 772.91it/s]

state: 10, average 0.0012546491622924805


## Check 12/41.4

In [64]:
import pickle
import os
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTStateV2
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
# cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import argparse
import copy
from perses.dispersed import feptasks
import mdtraj as md
import numpy as np
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from openmmtools.integrators import LangevinIntegrator
import time
from tqdm import tqdm

In [65]:
# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)


In [66]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/12/41/"
resid = '29'
old_aa_name = 'ALA'
new_aa_name = 'TYR'
sim_number = 4
length = 1
move_length = 3
timestep = 4
radius = 0.2

if sim_number == 1:
    phase = 'apo'
    name = old_aa_name
    state = 0
elif sim_number == 2:
    phase = 'apo'
    name = new_aa_name
    state = 1
elif sim_number == 3:
    phase = 'complex'
    name = old_aa_name
    state = 0
elif sim_number == 4:
    phase = 'complex'
    name = new_aa_name
    state = 1



In [67]:
i = os.path.basename(os.path.dirname(outdir))
path = os.path.join(outdir, f"{i}_{phase}_{state}.pickle")
htf = pickle.load(open(path, "rb" ))


In [68]:
# Build REST factory
_logger.info("Generating REST factory")
_logger.info(f"radius:{radius} nm")
query_indices = [atom.index for atom in list(htf.hybrid_topology.residues)[int(resid)].atoms]
_logger.info(f"query indices {query_indices}")
traj = md.Trajectory(np.array(htf.hybrid_positions), htf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=solute_atoms)[0])
factory = RESTTopologyFactory(htf.hybrid_system, solute_region=rest_atoms)

_logger.info("Generating REST states")
# Get REST system
REST_system = factory.REST_system

# Create states for each replica
n_replicas = 12  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 1200 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

# Create reference thermodynamic state
lambda_zero_alchemical_state = RESTStateV2.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

context_cache = cache.ContextCache()

# Create thermodynamics states
sampler_state =  SamplerState(htf.hybrid_positions, box_vectors=htf.hybrid_system.getDefaultPeriodicBoxVectors())
beta_0 = 1/(kB*T_min)
thermodynamic_state_list = []
sampler_state_list = []
for temperature in temperatures:
    beta_m = 1/(kB*temperature)
    compound_thermodynamic_state_copy = copy.deepcopy(compound_thermodynamic_state)
    compound_thermodynamic_state_copy.set_alchemical_parameters(beta_0, beta_m)
    thermodynamic_state_list.append(compound_thermodynamic_state_copy)

    # now generating a sampler_state for each thermodynamic state, with relaxed positions
    # context, context_integrator = context_cache.get_context(compound_thermodynamic_state_copy)
    feptasks.minimize(compound_thermodynamic_state_copy, sampler_state)
    sampler_state_list.append(copy.deepcopy(sampler_state))

INFO:root:Generating REST factory
INFO:root:radius:0.2 nm
INFO:root:query indices [466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 29466, 29467, 29468, 29469, 29470, 29471, 29472, 29473, 29474, 29475, 29476, 29477, 29478, 29479, 29480]
INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=6.7997, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=6.7997, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=6.7997), unit=nanometer)]
INFO:REST:No unknown forces.
INFO:root:Generating REST states


In [69]:
# Define simulation parameters
temperature = 298 * unit.kelvin
nsteps = 250 # 1 ps
timestep = 4.0 * unit.femtosecond
platform_name = 'CUDA'
collision_rate = 1.0 / unit.picoseconds

In [70]:
# Set up integrator
integrator = LangevinIntegrator(temperature, collision_rate, timestep)

# Set up context
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')

context = thermodynamic_state_list[11].create_context(integrator, platform=platform)    

context.setPeriodicBoxVectors(*htf.hybrid_system.getDefaultPeriodicBoxVectors())
context.setPositions(htf.hybrid_positions)
context.setVelocitiesToTemperature(temperature)
print(compound_thermodynamic_state.reduced_potential(context))

# Minimize
openmm.LocalEnergyMinimizer.minimize(context)



1213403.7042491394


In [71]:
# Run equilibration
times = []
for step in tqdm(range(nsteps)):
    initial_time = time.time()
    integrator.step(1)
    elapsed_time = (time.time() - initial_time) * unit.seconds
    print(f'took: {elapsed_time / unit.seconds} seconds')
    times.append(elapsed_time / unit.seconds)


 54%|█████▍    | 136/250 [00:00<00:00, 650.17it/s]

took: 0.002532482147216797 seconds
took: 0.0008547306060791016 seconds
took: 0.0010368824005126953 seconds
took: 0.0010576248168945312 seconds
took: 0.0013179779052734375 seconds
took: 0.0010585784912109375 seconds
took: 0.0010700225830078125 seconds
took: 0.0010938644409179688 seconds
took: 0.0011284351348876953 seconds
took: 0.0011086463928222656 seconds
took: 0.001135110855102539 seconds
took: 0.0013146400451660156 seconds
took: 0.0010771751403808594 seconds
took: 0.0011267662048339844 seconds
took: 0.0010824203491210938 seconds
took: 0.0011310577392578125 seconds
took: 0.0011260509490966797 seconds
took: 0.0013930797576904297 seconds
took: 0.0011227130889892578 seconds
took: 0.0010819435119628906 seconds
took: 0.0011692047119140625 seconds
took: 0.001134634017944336 seconds
took: 0.0010976791381835938 seconds
took: 0.001329660415649414 seconds
took: 0.001100301742553711 seconds
took: 0.001148223876953125 seconds
took: 0.0011606216430664062 seconds
took: 0.0011560916900634766 second

100%|██████████| 250/250 [00:00<00:00, 699.48it/s]


took: 0.0005290508270263672 seconds
took: 0.0009865760803222656 seconds
took: 0.0010488033294677734 seconds
took: 0.0010061264038085938 seconds
took: 0.0010530948638916016 seconds
took: 0.0012614727020263672 seconds
took: 0.0010080337524414062 seconds
took: 0.0010631084442138672 seconds
took: 0.0010156631469726562 seconds
took: 0.001046895980834961 seconds
took: 0.0012547969818115234 seconds
took: 0.0010645389556884766 seconds
took: 0.0036029815673828125 seconds
took: 0.00600123405456543 seconds
took: 0.0014371871948242188 seconds
took: 0.001009225845336914 seconds
took: 0.0010175704956054688 seconds
took: 0.001058816909790039 seconds
took: 0.0009911060333251953 seconds
took: 0.0012691020965576172 seconds
took: 0.0010001659393310547 seconds
took: 0.0010559558868408203 seconds
took: 0.001050710678100586 seconds
took: 0.0010476112365722656 seconds
took: 0.0012600421905517578 seconds
took: 0.0010523796081542969 seconds
took: 0.001071929931640625 seconds
took: 0.0010101795196533203 second

In [54]:
print(f"average {np.mean(times)}")

average 0.0012101144790649413


## Run repex for 12/41.1

In [81]:
import pickle
import os
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTStateV2
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
# cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import argparse
import copy
from perses.dispersed import feptasks
import mdtraj as md
import numpy as np
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from openmmtools.integrators import LangevinIntegrator
import time
from tqdm import tqdm

In [82]:
# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)


In [88]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/12/41/"
resid = '29'
old_aa_name = 'ALA'
new_aa_name = 'TYR'
sim_number = 1
length = 1
move_length = 3
timestep = 4
radius = 0.2

if sim_number == 1:
    phase = 'apo'
    name = old_aa_name
    state = 0
elif sim_number == 2:
    phase = 'apo'
    name = new_aa_name
    state = 1
elif sim_number == 3:
    phase = 'complex'
    name = old_aa_name
    state = 0
elif sim_number == 4:
    phase = 'complex'
    name = new_aa_name
    state = 1



In [89]:
i = os.path.basename(os.path.dirname(outdir))
path = os.path.join(outdir, f"{i}_{phase}_{state}.pickle")
htf = pickle.load(open(path, "rb" ))


In [90]:
# Build REST factory
_logger.info("Generating REST factory")
_logger.info(f"radius:{radius} nm")
query_indices = [atom.index for atom in list(htf.hybrid_topology.residues)[int(resid)].atoms]
_logger.info(f"query indices {query_indices}")
traj = md.Trajectory(np.array(htf.hybrid_positions), htf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=solute_atoms)[0])
factory = RESTTopologyFactory(htf.hybrid_system, solute_region=rest_atoms)

_logger.info("Generating REST states")
# Get REST system
REST_system = factory.REST_system

# Create states for each replica
n_replicas = 12  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 1200 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

# Create reference thermodynamic state
lambda_zero_alchemical_state = RESTStateV2.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

context_cache = cache.ContextCache()

# Create thermodynamics states
sampler_state =  SamplerState(htf.hybrid_positions, box_vectors=htf.hybrid_system.getDefaultPeriodicBoxVectors())
beta_0 = 1/(kB*T_min)
thermodynamic_state_list = []
sampler_state_list = []
for temperature in temperatures:
    beta_m = 1/(kB*temperature)
    compound_thermodynamic_state_copy = copy.deepcopy(compound_thermodynamic_state)
    compound_thermodynamic_state_copy.set_alchemical_parameters(beta_0, beta_m)
    thermodynamic_state_list.append(compound_thermodynamic_state_copy)

    # now generating a sampler_state for each thermodynamic state, with relaxed positions
    # context, context_integrator = context_cache.get_context(compound_thermodynamic_state_copy)
    feptasks.minimize(compound_thermodynamic_state_copy, sampler_state)
    sampler_state_list.append(copy.deepcopy(sampler_state))

INFO:root:Generating REST factory
INFO:root:radius:0.2 nm
INFO:root:query indices [466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 14881, 14882, 14883, 14884, 14885, 14886, 14887, 14888, 14889, 14890, 14891, 14892, 14893, 14894, 14895]
INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=5.423900000000001, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=5.423900000000001, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=5.423900000000001), unit=nanometer)]
INFO:REST:No unknown forces.
INFO:root:Generating REST states


In [91]:
# Set up sampler
_logger.setLevel(logging.DEBUG)
_logger.info("About to start repex")
print(f"move steps: {int((move_length*1000)/timestep)}")
print(f"timestep: {timestep} fs")
move = mcmc.LangevinSplittingDynamicsMove(timestep=timestep*unit.femtoseconds, n_steps=int((move_length*1000)/timestep))
simulation = multistate.ReplicaExchangeSampler(mcmc_moves=move, number_of_iterations=length*1000)

# Run t-repex
reporter_file = os.path.join(outdir, f"{i}_{phase}_{name.lower()}_{length}ns_test.nc")
reporter = multistate.MultiStateReporter(reporter_file, checkpoint_interval=1)
simulation.create(thermodynamic_states=thermodynamic_state_list,
                  sampler_states=sampler_state_list,
                  storage=reporter)
simulation.run()

INFO:root:About to start repex
DEBUG:openmmtools.multistate.multistatereporter:Initial checkpoint file automatically chosen as /data/chodera/zhangi/perses_benchmark/neq/12/41/41_apo_ala_1ns_test_checkpoint.nc
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.storage_exists of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2af4555c5d90>>
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._display_citations at 0x2af3cd2e4d40>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._display_citations at 0x2af3cd296dd0>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._initialize_reporter at 0x2af3cd2990e0>


move steps: 750
timestep: 4 fs
Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys.,

DEBUG:openmmtools.multistate.multistatereporter:Serialized state thermodynamic_states/0 is  248240B | 242.422KB | 0.237MB
DEBUG:openmmtools.utils:Storing thermodynamic states took    0.534s
DEBUG:openmmtools.multistate.multistatesampler:Storing general ReplicaExchange options...
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2af3cd299200>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2af3cd2994d0>
DEBUG:openmmtools.utils:Storing sampler states took    0.007s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.081s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.463s
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.write_energies of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2af4555c5d90>>
DEBUG:openmmtools.multistate.multistat

DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:openmmtools.multistate.multistatesampler:Iteration 4/1000
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2af3cd2d3d40>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.001s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 10594/41472 attempted swaps (25.5%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took    7.694s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:

KeyboardInterrupt: 

## Run repex for 12/45.1

In [92]:
import pickle
import os
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTStateV2
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
# cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import argparse
import copy
from perses.dispersed import feptasks
import mdtraj as md
import numpy as np
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from openmmtools.integrators import LangevinIntegrator
import time
from tqdm import tqdm

In [93]:
# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)


In [94]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/12/45/"
resid = '44'
old_aa_name = 'PHE'
new_aa_name = 'TRP'
sim_number = 1
length = 1
move_length = 3
timestep = 4
radius = 0.2

if sim_number == 1:
    phase = 'apo'
    name = old_aa_name
    state = 0
elif sim_number == 2:
    phase = 'apo'
    name = new_aa_name
    state = 1
elif sim_number == 3:
    phase = 'complex'
    name = old_aa_name
    state = 0
elif sim_number == 4:
    phase = 'complex'
    name = new_aa_name
    state = 1



In [95]:
i = os.path.basename(os.path.dirname(outdir))
path = os.path.join(outdir, f"{i}_{phase}_{state}.pickle")
htf = pickle.load(open(path, "rb" ))


In [96]:
# Build REST factory
_logger.info("Generating REST factory")
_logger.info(f"radius:{radius} nm")
query_indices = [atom.index for atom in list(htf.hybrid_topology.residues)[int(resid)].atoms]
_logger.info(f"query indices {query_indices}")
traj = md.Trajectory(np.array(htf.hybrid_positions), htf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=solute_atoms)[0])
factory = RESTTopologyFactory(htf.hybrid_system, solute_region=rest_atoms)

_logger.info("Generating REST states")
# Get REST system
REST_system = factory.REST_system

# Create states for each replica
n_replicas = 12  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 1200 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

# Create reference thermodynamic state
lambda_zero_alchemical_state = RESTStateV2.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

context_cache = cache.ContextCache()

# Create thermodynamics states
sampler_state =  SamplerState(htf.hybrid_positions, box_vectors=htf.hybrid_system.getDefaultPeriodicBoxVectors())
beta_0 = 1/(kB*T_min)
thermodynamic_state_list = []
sampler_state_list = []
for temperature in temperatures:
    beta_m = 1/(kB*temperature)
    compound_thermodynamic_state_copy = copy.deepcopy(compound_thermodynamic_state)
    compound_thermodynamic_state_copy.set_alchemical_parameters(beta_0, beta_m)
    thermodynamic_state_list.append(compound_thermodynamic_state_copy)

    # now generating a sampler_state for each thermodynamic state, with relaxed positions
    # context, context_integrator = context_cache.get_context(compound_thermodynamic_state_copy)
    feptasks.minimize(compound_thermodynamic_state_copy, sampler_state)
    sampler_state_list.append(copy.deepcopy(sampler_state))

INFO:root:Generating REST factory
INFO:root:radius:0.2 nm
INFO:root:query indices [690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 14906, 14907, 14908, 14909, 14910, 14911, 14912, 14913, 14914, 14915, 14916, 14917, 14918, 14919, 14920, 14921, 14922, 14923]
INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=5.423900000000001, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=5.423900000000001, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=5.423900000000001), unit=nanometer)]
INFO:REST:No unknown forces.
INFO:root:Generating REST states


In [97]:
# Set up sampler
_logger.setLevel(logging.DEBUG)
_logger.info("About to start repex")
print(f"move steps: {int((move_length*1000)/timestep)}")
print(f"timestep: {timestep} fs")
move = mcmc.LangevinSplittingDynamicsMove(timestep=timestep*unit.femtoseconds, n_steps=int((move_length*1000)/timestep))
simulation = multistate.ReplicaExchangeSampler(mcmc_moves=move, number_of_iterations=length*1000)

# Run t-repex
reporter_file = os.path.join(outdir, f"{i}_{phase}_{name.lower()}_{length}ns_test.nc")
reporter = multistate.MultiStateReporter(reporter_file, checkpoint_interval=1)
simulation.create(thermodynamic_states=thermodynamic_state_list,
                  sampler_states=sampler_state_list,
                  storage=reporter)
simulation.run()

INFO:root:About to start repex
DEBUG:openmmtools.multistate.multistatereporter:Initial checkpoint file automatically chosen as /data/chodera/zhangi/perses_benchmark/neq/12/45/45_apo_phe_1ns_test_checkpoint.nc
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.storage_exists of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2af45a27d8d0>>
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._display_citations at 0x2af3cd2e4d40>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._display_citations at 0x2af3cd296dd0>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._initialize_reporter at 0x2af3cd2990e0>


move steps: 750
timestep: 4 fs
Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys.,

DEBUG:openmmtools.multistate.multistatereporter:Serialized state thermodynamic_states/0 is  250005B | 244.146KB | 0.238MB
DEBUG:openmmtools.utils:Storing thermodynamic states took    0.568s
DEBUG:openmmtools.multistate.multistatesampler:Storing general ReplicaExchange options...
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2af3cd299200>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2af3cd2994d0>
DEBUG:openmmtools.utils:Storing sampler states took    0.007s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.090s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.548s
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.write_energies of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2af45a27d8d0>>
DEBUG:openmmtools.multistate.multistat

DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:openmmtools.multistate.multistatesampler:Iteration 4/1000
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2af3cd2d3d40>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.001s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 6472/41472 attempted swaps (15.6%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.


KeyboardInterrupt: 

## Run repex for 12/47.1

In [1]:
import pickle
import os
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTStateV2
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
# cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import argparse
import copy
from perses.dispersed import feptasks
import mdtraj as md
import numpy as np
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from openmmtools.integrators import LangevinIntegrator
import time
from tqdm import tqdm

INFO:numexpr.utils:Note: NumExpr detected 48 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


conducting subsequent work with the following platform: CUDA


INFO:rdkit:Enabling RDKit 2020.09.1 jupyter extensions


In [2]:
# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)


In [3]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/12/47/"
resid = '29'
old_aa_name = 'PHE'
new_aa_name = 'TYR'
sim_number = 1
length = 1
move_length = 3
timestep = 4
radius = 0.2

if sim_number == 1:
    phase = 'apo'
    name = old_aa_name
    state = 0
elif sim_number == 2:
    phase = 'apo'
    name = new_aa_name
    state = 1
elif sim_number == 3:
    phase = 'complex'
    name = old_aa_name
    state = 0
elif sim_number == 4:
    phase = 'complex'
    name = new_aa_name
    state = 1



In [4]:
i = os.path.basename(os.path.dirname(outdir))
path = os.path.join(outdir, f"{i}_{phase}_{state}.pickle")
htf = pickle.load(open(path, "rb" ))


In [5]:
# Build REST factory
_logger.info("Generating REST factory")
_logger.info(f"radius:{radius} nm")
query_indices = [atom.index for atom in list(htf.hybrid_topology.residues)[int(resid)].atoms]
_logger.info(f"query indices {query_indices}")
traj = md.Trajectory(np.array(htf.hybrid_positions), htf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=solute_atoms)[0])
factory = RESTTopologyFactory(htf.hybrid_system, solute_region=rest_atoms)

_logger.info("Generating REST states")
# Get REST system
REST_system = factory.REST_system

# Create states for each replica
n_replicas = 12  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 1200 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

# Create reference thermodynamic state
lambda_zero_alchemical_state = RESTStateV2.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

context_cache = cache.ContextCache()

# Create thermodynamics states
sampler_state =  SamplerState(htf.hybrid_positions, box_vectors=htf.hybrid_system.getDefaultPeriodicBoxVectors())
beta_0 = 1/(kB*T_min)
thermodynamic_state_list = []
sampler_state_list = []
for temperature in temperatures:
    beta_m = 1/(kB*temperature)
    compound_thermodynamic_state_copy = copy.deepcopy(compound_thermodynamic_state)
    compound_thermodynamic_state_copy.set_alchemical_parameters(beta_0, beta_m)
    thermodynamic_state_list.append(compound_thermodynamic_state_copy)

    # now generating a sampler_state for each thermodynamic state, with relaxed positions
    # context, context_integrator = context_cache.get_context(compound_thermodynamic_state_copy)
    feptasks.minimize(compound_thermodynamic_state_copy, sampler_state)
    sampler_state_list.append(copy.deepcopy(sampler_state))

INFO:root:Generating REST factory
INFO:root:radius:0.2 nm
INFO:root:query indices [466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 14879, 14880, 14881]
INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=5.423900000000001, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=5.423900000000001, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=5.423900000000001), unit=nanometer)]
INFO:REST:No unknown forces.
INFO:root:Generating REST states


In [6]:
# Set up sampler
_logger.setLevel(logging.DEBUG)
_logger.info("About to start repex")
print(f"move steps: {int((move_length*1000)/timestep)}")
print(f"timestep: {timestep} fs")
move = mcmc.LangevinSplittingDynamicsMove(timestep=timestep*unit.femtoseconds, n_steps=int((move_length*1000)/timestep))
simulation = multistate.ReplicaExchangeSampler(mcmc_moves=move, number_of_iterations=length*1000)

# Run t-repex
reporter_file = os.path.join(outdir, f"{i}_{phase}_{name.lower()}_{length}ns_test.nc")
reporter = multistate.MultiStateReporter(reporter_file, checkpoint_interval=1)
simulation.create(thermodynamic_states=thermodynamic_state_list,
                  sampler_states=sampler_state_list,
                  storage=reporter)
simulation.run()

INFO:root:About to start repex
DEBUG:openmmtools.multistate.multistatereporter:Initial checkpoint file automatically chosen as /data/chodera/zhangi/perses_benchmark/neq/12/47/47_apo_phe_1ns_test_checkpoint.nc
DEBUG:mpiplus.mpiplus:Cannot find MPI environment. MPI disabled.
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.storage_exists of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2b817e77ead0>>
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._display_citations at 0x2b8144632dd0>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._display_citations at 0x2b81445e0e60>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._initialize_reporter at 0x2b81445e6170>


move steps: 750
timestep: 4 fs
Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys.,

DEBUG:openmmtools.multistate.multistatereporter:Serialized state thermodynamic_states/0 is  248251B | 242.433KB | 0.237MB
DEBUG:openmmtools.utils:Storing thermodynamic states took    0.663s
DEBUG:openmmtools.multistate.multistatesampler:Storing general ReplicaExchange options...
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2b81445e6290>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2b81445e6560>
DEBUG:openmmtools.utils:Storing sampler states took    0.012s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.105s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.489s
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.write_energies of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2b817e77ead0>>
DEBUG:openmmtools.multistate.multistat

KeyboardInterrupt: 

## Run repex for 12/43.2

In [2]:
import pickle
import os
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTStateV2
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import argparse
import copy
from perses.dispersed import feptasks
import mdtraj as md
import numpy as np
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from openmmtools.integrators import LangevinIntegrator
import time
from tqdm import tqdm

conducting subsequent work with the following platform: CUDA


In [3]:
# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)


In [4]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/12/43/"
resid = '38'
old_aa_name = 'PHE'
new_aa_name = 'TRP'
sim_number = 2
length = 1
move_length = 3
timestep = 4
radius = 0.2

if sim_number == 1:
    phase = 'apo'
    name = old_aa_name
    state = 0
elif sim_number == 2:
    phase = 'apo'
    name = new_aa_name
    state = 1
elif sim_number == 3:
    phase = 'complex'
    name = old_aa_name
    state = 0
elif sim_number == 4:
    phase = 'complex'
    name = new_aa_name
    state = 1



In [5]:
i = os.path.basename(os.path.dirname(outdir))
path = os.path.join(outdir, f"{i}_{phase}_{state}.pickle")
htf = pickle.load(open(path, "rb" ))


In [6]:
# Build REST factory
_logger.info("Generating REST factory")
_logger.info(f"radius:{radius} nm")
query_indices = [atom.index for atom in list(htf.hybrid_topology.residues)[int(resid)].atoms]
_logger.info(f"query indices {query_indices}")
traj = md.Trajectory(np.array(htf.hybrid_positions), htf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=solute_atoms)[0])
factory = RESTTopologyFactory(htf.hybrid_system, solute_region=rest_atoms)

_logger.info("Generating REST states")
# Get REST system
REST_system = factory.REST_system

# Create states for each replica
n_replicas = 12  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 1200 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

# Create reference thermodynamic state
lambda_zero_alchemical_state = RESTStateV2.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

context_cache = cache.ContextCache()

# Create thermodynamics states
sampler_state =  SamplerState(htf.hybrid_positions, box_vectors=htf.hybrid_system.getDefaultPeriodicBoxVectors())
beta_0 = 1/(kB*T_min)
thermodynamic_state_list = []
sampler_state_list = []
for temperature in temperatures:
    beta_m = 1/(kB*temperature)
    compound_thermodynamic_state_copy = copy.deepcopy(compound_thermodynamic_state)
    compound_thermodynamic_state_copy.set_alchemical_parameters(beta_0, beta_m)
    thermodynamic_state_list.append(compound_thermodynamic_state_copy)

    # now generating a sampler_state for each thermodynamic state, with relaxed positions
    # context, context_integrator = context_cache.get_context(compound_thermodynamic_state_copy)
    feptasks.minimize(compound_thermodynamic_state_copy, sampler_state)
    sampler_state_list.append(copy.deepcopy(sampler_state))

INFO:root:Generating REST factory
INFO:root:radius:0.2 nm
INFO:root:query indices [604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 14876, 14877, 14878, 14879, 14880, 14881, 14882, 14883, 14884, 14885, 14886, 14887, 14888, 14889, 14890, 14891, 14892, 14893]
INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=5.423900000000001, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=5.423900000000001, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=5.423900000000001), unit=nanometer)]
INFO:REST:No unknown forces.
INFO:root:Generating REST states


In [7]:
# Set up sampler
_logger.setLevel(logging.DEBUG)
_logger.info("About to start repex")
print(f"move steps: {int((move_length*1000)/timestep)}")
print(f"timestep: {timestep} fs")
move = mcmc.LangevinSplittingDynamicsMove(timestep=timestep*unit.femtoseconds, n_steps=int((move_length*1000)/timestep))
simulation = multistate.ReplicaExchangeSampler(mcmc_moves=move, number_of_iterations=length*1000)

# Run t-repex
reporter_file = os.path.join(outdir, f"{i}_{phase}_{name.lower()}_{length}ns_test.nc")
reporter = multistate.MultiStateReporter(reporter_file, checkpoint_interval=1)
simulation.create(thermodynamic_states=thermodynamic_state_list,
                  sampler_states=sampler_state_list,
                  storage=reporter)
simulation.run()

INFO:root:About to start repex
DEBUG:openmmtools.multistate.multistatereporter:Initial checkpoint file automatically chosen as /data/chodera/zhangi/perses_benchmark/neq/12/43/43_apo_trp_1ns_test_checkpoint.nc
DEBUG:mpiplus.mpiplus:Cannot find MPI environment. MPI disabled.
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.storage_exists of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2b42f86b5b50>>
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._display_citations at 0x2b42ae51dcb0>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._display_citations at 0x2b42ae4cdd40>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._initialize_reporter at 0x2b42ae4d0050>


move steps: 750
timestep: 4 fs
Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys.,

DEBUG:openmmtools.multistate.multistatereporter:Serialized state thermodynamic_states/0 is  251352B | 245.461KB | 0.240MB
DEBUG:openmmtools.utils:Storing thermodynamic states took    0.519s
DEBUG:openmmtools.multistate.multistatesampler:Storing general ReplicaExchange options...
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2b42ae4d0170>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2b42ae4d0440>
DEBUG:openmmtools.utils:Storing sampler states took    0.006s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.079s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.135s
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.write_energies of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2b42f86b5b50>>
DEBUG:openmmtools.multistate.multistat

DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:openmmtools.multistate.multistatesampler:Iteration 4/1000
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2b42ae509cb0>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.001s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 6892/41472 attempted swaps (16.6%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took    7.666s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:C

DEBUG:openmmtools.multistate.multistatesampler:*** ONLINE analysis free energies:
DEBUG:openmmtools.multistate.multistatesampler:        -0.0     0.3     0.3     0.4     0.4     0.4     0.4     0.3     0.2    -0.1    -1.0   -15.9
DEBUG:openmmtools.utils:Computing online free energy estimate took    0.005s
DEBUG:openmmtools.multistate.multistatesampler:Iteration took 8.025s.
DEBUG:openmmtools.multistate.multistatesampler:Estimated completion in 2:13:12.606597, at Wed Jan 20 15:08:36 2021 (consuming total wall clock time 2:14:08.949242).
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:openmmtools.multistate.multistatesampler:Iteration 8/1000
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2b42ae509cb0>
DEBUG:openmmtools.multistat

DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2b42ae4d0440>
DEBUG:openmmtools.utils:Storing sampler states took    0.004s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.070s
DEBUG:openmmtools.multistate.multistatesampler:Not enough iterations for online analysis (self.online_analysis_minimum_iterations = 200)
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._online_analysis at 0x2b42ae4d0f80>
DEBUG:openmmtools.multistate.multistatesampler:*** ONLINE analysis free energies:
DEBUG:openmmtools.multistate.multistatesampler:        -0.0     0.4     0.5     0.5     0.5     0.6     0.5     0.5     0.3     0.1    -1.1   -19.2
DEBUG:openmmtools.utils:Computing online free energy estimate took    0.005s
DEBUG:openmmtools.multistate.multistatesampler:Iteration took 8.176s.
DEBUG:openmmtools.multistate.multistatesampler:Estimated completion in 2:13:09.217051, at Wed Jan 20 15:09:05 2021 (con

DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took    8.150s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.150s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2b42ae4d0170>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2b42ae4d0440>
DEBUG:openmmtools.utils:Storing sampler states took    0.006s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.073s
DEBUG:openmmtools.multistate.multistatesampler:Not enough iterations for online analysis (self.online_analysis_minimum_iterations = 200)
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._online_analysis at 0x2b42

DEBUG:openmmtools.multistate.multistatesampler:Iteration 19/1000
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2b42ae509cb0>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.001s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 6076/41472 attempted swaps (14.7%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took    8.285s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.153s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteratio

Traceback (most recent call last):
  File "/home/zhangi/miniconda3/envs/perses-sims-oct2020/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-a2f3a34d13d5>", line 15, in <module>
    simulation.run()
  File "/home/zhangi/miniconda3/envs/perses-sims-oct2020/lib/python3.7/site-packages/openmmtools-0.20.0+19.g27982a0.dirty-py3.7-linux-x86_64.egg/openmmtools/multistate/multistatesampler.py", line 681, in run
    self._propagate_replicas()
  File "/home/zhangi/miniconda3/envs/perses-sims-oct2020/lib/python3.7/site-packages/openmmtools-0.20.0+19.g27982a0.dirty-py3.7-linux-x86_64.egg/openmmtools/utils.py", line 87, in _wrapper
    return func(*args, **kwargs)
  File "/home/zhangi/miniconda3/envs/perses-sims-oct2020/lib/python3.7/site-packages/openmmtools-0.20.0+19.g27982a0.dirty-py3.7-linux-x86_64.egg/openmmtools/multistate/multistatesampler.py", line 1197, in _propagate_replicas

TypeError: object of type 'NoneType' has no len()

In [14]:
for atom in htf._topology_proposal.new_topology.atoms():
    print(atom)

<Atom 0 (CH3) of chain 0 residue 0 (ACE)>
<Atom 1 (C) of chain 0 residue 0 (ACE)>
<Atom 2 (O) of chain 0 residue 0 (ACE)>
<Atom 3 (H1) of chain 0 residue 0 (ACE)>
<Atom 4 (H2) of chain 0 residue 0 (ACE)>
<Atom 5 (H3) of chain 0 residue 0 (ACE)>
<Atom 6 (N) of chain 0 residue 1 (LYS)>
<Atom 7 (CA) of chain 0 residue 1 (LYS)>
<Atom 8 (C) of chain 0 residue 1 (LYS)>
<Atom 9 (O) of chain 0 residue 1 (LYS)>
<Atom 10 (CB) of chain 0 residue 1 (LYS)>
<Atom 11 (CG) of chain 0 residue 1 (LYS)>
<Atom 12 (CD) of chain 0 residue 1 (LYS)>
<Atom 13 (CE) of chain 0 residue 1 (LYS)>
<Atom 14 (NZ) of chain 0 residue 1 (LYS)>
<Atom 15 (H2) of chain 0 residue 1 (LYS)>
<Atom 16 (HA) of chain 0 residue 1 (LYS)>
<Atom 17 (HB3) of chain 0 residue 1 (LYS)>
<Atom 18 (HB2) of chain 0 residue 1 (LYS)>
<Atom 19 (HG2) of chain 0 residue 1 (LYS)>
<Atom 20 (HG3) of chain 0 residue 1 (LYS)>
<Atom 21 (HD2) of chain 0 residue 1 (LYS)>
<Atom 22 (HD3) of chain 0 residue 1 (LYS)>
<Atom 23 (HE3) of chain 0 residue 1 (LYS)>

<Atom 3059 (H1) of chain 2 residue 630 (HOH)>
<Atom 3060 (H2) of chain 2 residue 630 (HOH)>
<Atom 3061 (O) of chain 2 residue 631 (HOH)>
<Atom 3062 (H1) of chain 2 residue 631 (HOH)>
<Atom 3063 (H2) of chain 2 residue 631 (HOH)>
<Atom 3064 (O) of chain 2 residue 632 (HOH)>
<Atom 3065 (H1) of chain 2 residue 632 (HOH)>
<Atom 3066 (H2) of chain 2 residue 632 (HOH)>
<Atom 3067 (O) of chain 2 residue 633 (HOH)>
<Atom 3068 (H1) of chain 2 residue 633 (HOH)>
<Atom 3069 (H2) of chain 2 residue 633 (HOH)>
<Atom 3070 (O) of chain 2 residue 634 (HOH)>
<Atom 3071 (H1) of chain 2 residue 634 (HOH)>
<Atom 3072 (H2) of chain 2 residue 634 (HOH)>
<Atom 3073 (O) of chain 2 residue 635 (HOH)>
<Atom 3074 (H1) of chain 2 residue 635 (HOH)>
<Atom 3075 (H2) of chain 2 residue 635 (HOH)>
<Atom 3076 (O) of chain 2 residue 636 (HOH)>
<Atom 3077 (H1) of chain 2 residue 636 (HOH)>
<Atom 3078 (H2) of chain 2 residue 636 (HOH)>
<Atom 3079 (O) of chain 2 residue 637 (HOH)>
<Atom 3080 (H1) of chain 2 residue 637 (H

<Atom 6559 (O) of chain 2 residue 1797 (HOH)>
<Atom 6560 (H1) of chain 2 residue 1797 (HOH)>
<Atom 6561 (H2) of chain 2 residue 1797 (HOH)>
<Atom 6562 (O) of chain 2 residue 1798 (HOH)>
<Atom 6563 (H1) of chain 2 residue 1798 (HOH)>
<Atom 6564 (H2) of chain 2 residue 1798 (HOH)>
<Atom 6565 (O) of chain 2 residue 1799 (HOH)>
<Atom 6566 (H1) of chain 2 residue 1799 (HOH)>
<Atom 6567 (H2) of chain 2 residue 1799 (HOH)>
<Atom 6568 (O) of chain 2 residue 1800 (HOH)>
<Atom 6569 (H1) of chain 2 residue 1800 (HOH)>
<Atom 6570 (H2) of chain 2 residue 1800 (HOH)>
<Atom 6571 (O) of chain 2 residue 1801 (HOH)>
<Atom 6572 (H1) of chain 2 residue 1801 (HOH)>
<Atom 6573 (H2) of chain 2 residue 1801 (HOH)>
<Atom 6574 (O) of chain 2 residue 1802 (HOH)>
<Atom 6575 (H1) of chain 2 residue 1802 (HOH)>
<Atom 6576 (H2) of chain 2 residue 1802 (HOH)>
<Atom 6577 (O) of chain 2 residue 1803 (HOH)>
<Atom 6578 (H1) of chain 2 residue 1803 (HOH)>
<Atom 6579 (H2) of chain 2 residue 1803 (HOH)>
<Atom 6580 (O) of ch

<Atom 10058 (H1) of chain 2 residue 2963 (HOH)>
<Atom 10059 (H2) of chain 2 residue 2963 (HOH)>
<Atom 10060 (O) of chain 2 residue 2964 (HOH)>
<Atom 10061 (H1) of chain 2 residue 2964 (HOH)>
<Atom 10062 (H2) of chain 2 residue 2964 (HOH)>
<Atom 10063 (O) of chain 2 residue 2965 (HOH)>
<Atom 10064 (H1) of chain 2 residue 2965 (HOH)>
<Atom 10065 (H2) of chain 2 residue 2965 (HOH)>
<Atom 10066 (O) of chain 2 residue 2966 (HOH)>
<Atom 10067 (H1) of chain 2 residue 2966 (HOH)>
<Atom 10068 (H2) of chain 2 residue 2966 (HOH)>
<Atom 10069 (O) of chain 2 residue 2967 (HOH)>
<Atom 10070 (H1) of chain 2 residue 2967 (HOH)>
<Atom 10071 (H2) of chain 2 residue 2967 (HOH)>
<Atom 10072 (O) of chain 2 residue 2968 (HOH)>
<Atom 10073 (H1) of chain 2 residue 2968 (HOH)>
<Atom 10074 (H2) of chain 2 residue 2968 (HOH)>
<Atom 10075 (O) of chain 2 residue 2969 (HOH)>
<Atom 10076 (H1) of chain 2 residue 2969 (HOH)>
<Atom 10077 (H2) of chain 2 residue 2969 (HOH)>
<Atom 10078 (O) of chain 2 residue 2970 (HOH)>

<Atom 14558 (H1) of chain 2 residue 4463 (HOH)>
<Atom 14559 (H2) of chain 2 residue 4463 (HOH)>
<Atom 14560 (O) of chain 2 residue 4464 (HOH)>
<Atom 14561 (H1) of chain 2 residue 4464 (HOH)>
<Atom 14562 (H2) of chain 2 residue 4464 (HOH)>
<Atom 14563 (O) of chain 2 residue 4465 (HOH)>
<Atom 14564 (H1) of chain 2 residue 4465 (HOH)>
<Atom 14565 (H2) of chain 2 residue 4465 (HOH)>
<Atom 14566 (O) of chain 2 residue 4466 (HOH)>
<Atom 14567 (H1) of chain 2 residue 4466 (HOH)>
<Atom 14568 (H2) of chain 2 residue 4466 (HOH)>
<Atom 14569 (O) of chain 2 residue 4467 (HOH)>
<Atom 14570 (H1) of chain 2 residue 4467 (HOH)>
<Atom 14571 (H2) of chain 2 residue 4467 (HOH)>
<Atom 14572 (O) of chain 2 residue 4468 (HOH)>
<Atom 14573 (H1) of chain 2 residue 4468 (HOH)>
<Atom 14574 (H2) of chain 2 residue 4468 (HOH)>
<Atom 14575 (O) of chain 2 residue 4469 (HOH)>
<Atom 14576 (H1) of chain 2 residue 4469 (HOH)>
<Atom 14577 (H2) of chain 2 residue 4469 (HOH)>
<Atom 14578 (O) of chain 2 residue 4470 (HOH)>

## Run repex for 12/43.1,2, 4

In [1]:
import pickle
import os
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTStateV2
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
# cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import argparse
import copy
from perses.dispersed import feptasks
import mdtraj as md
import numpy as np
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from openmmtools.integrators import LangevinIntegrator
import time
from tqdm import tqdm

INFO:numexpr.utils:Note: NumExpr detected 48 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


conducting subsequent work with the following platform: CUDA


INFO:rdkit:Enabling RDKit 2020.09.1 jupyter extensions


In [2]:
# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)


In [3]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/12/43/"
resid = '38'
old_aa_name = 'PHE'
new_aa_name = 'TRP'
sim_number = 4
length = 1
move_length = 3
timestep = 4
radius = 0.2

if sim_number == 1:
    phase = 'apo'
    name = old_aa_name
    state = 0
elif sim_number == 2:
    phase = 'apo'
    name = new_aa_name
    state = 1
elif sim_number == 3:
    phase = 'complex'
    name = old_aa_name
    state = 0
elif sim_number == 4:
    phase = 'complex'
    name = new_aa_name
    state = 1



In [4]:
i = os.path.basename(os.path.dirname(outdir))
path = os.path.join(outdir, f"{i}_{phase}_{state}.pickle")
htf = pickle.load(open(path, "rb" ))


In [5]:
# Build REST factory
_logger.info("Generating REST factory")
_logger.info(f"radius:{radius} nm")
query_indices = [atom.index for atom in list(htf.hybrid_topology.residues)[int(resid)].atoms]
_logger.info(f"query indices {query_indices}")
traj = md.Trajectory(np.array(htf.hybrid_positions), htf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=solute_atoms)[0])
factory = RESTTopologyFactory(htf.hybrid_system, solute_region=rest_atoms)

_logger.info("Generating REST states")
# Get REST system
REST_system = factory.REST_system

# Create states for each replica
n_replicas = 12  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 1200 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

# Create reference thermodynamic state
lambda_zero_alchemical_state = RESTStateV2.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

context_cache = cache.ContextCache()

# Create thermodynamics states
sampler_state =  SamplerState(htf.hybrid_positions, box_vectors=htf.hybrid_system.getDefaultPeriodicBoxVectors())
beta_0 = 1/(kB*T_min)
thermodynamic_state_list = []
sampler_state_list = []
for temperature in temperatures:
    beta_m = 1/(kB*temperature)
    compound_thermodynamic_state_copy = copy.deepcopy(compound_thermodynamic_state)
    compound_thermodynamic_state_copy.set_alchemical_parameters(beta_0, beta_m)
    thermodynamic_state_list.append(compound_thermodynamic_state_copy)

    # now generating a sampler_state for each thermodynamic state, with relaxed positions
    # context, context_integrator = context_cache.get_context(compound_thermodynamic_state_copy)
    feptasks.minimize(compound_thermodynamic_state_copy, sampler_state)
    sampler_state_list.append(copy.deepcopy(sampler_state))

INFO:root:Generating REST factory
INFO:root:radius:0.2 nm
INFO:root:query indices [604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 29473, 29474, 29475, 29476, 29477, 29478, 29479, 29480, 29481, 29482, 29483, 29484, 29485, 29486, 29487, 29488, 29489, 29490]
INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=6.7997, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=6.7997, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=6.7997), unit=nanometer)]
INFO:REST:No unknown forces.
INFO:root:Generating REST states


In [7]:
# Set up sampler
_logger.setLevel(logging.DEBUG)
_logger.info("About to start repex")
print(f"move steps: {int((move_length*1000)/timestep)}")
print(f"timestep: {timestep} fs")
move = mcmc.LangevinSplittingDynamicsMove(timestep=timestep*unit.femtoseconds, n_steps=int((move_length*1000)/timestep))
simulation = multistate.ReplicaExchangeSampler(mcmc_moves=move, number_of_iterations=length*1000)

# Run t-repex
reporter_file = os.path.join(outdir, f"{i}_{phase}_{name.lower()}_{length}ns_test.nc")
reporter = multistate.MultiStateReporter(reporter_file, checkpoint_interval=1)
simulation.create(thermodynamic_states=thermodynamic_state_list,
                  sampler_states=sampler_state_list,
                  storage=reporter)
simulation.run()

INFO:root:About to start repex
DEBUG:openmmtools.multistate.multistatereporter:Initial checkpoint file automatically chosen as /data/chodera/zhangi/perses_benchmark/neq/12/43/43_complex_trp_1ns_test_checkpoint.nc
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.storage_exists of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2b3c7ef41850>>
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._display_citations at 0x2b3c3954ecb0>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._display_citations at 0x2b3c394fdd40>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._initialize_reporter at 0x2b3c39500050>


move steps: 750
timestep: 4 fs
Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys.,

DEBUG:openmmtools.multistate.multistatereporter:Serialized state thermodynamic_states/0 is  506696B | 494.820KB | 0.483MB
DEBUG:openmmtools.utils:Storing thermodynamic states took    1.843s
DEBUG:openmmtools.multistate.multistatesampler:Storing general ReplicaExchange options...
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2b3c39500170>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2b3c39500440>
DEBUG:openmmtools.utils:Storing sampler states took    0.019s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.211s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.239s
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.write_energies of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2b3c7ef41850>>
DEBUG:openmmtools.multistate.multistat

SimulationNaNError: Propagating replica 1 at state 0 resulted in a NaN!
The state of the system and integrator before the error were saved in /data/chodera/zhangi/perses_benchmark/neq/12/43/nan-error-logs

In [14]:
for atom in htf._topology_proposal.new_topology.atoms():
    print(atom)

<Atom 0 (CH3) of chain 0 residue 0 (ACE)>
<Atom 1 (C) of chain 0 residue 0 (ACE)>
<Atom 2 (O) of chain 0 residue 0 (ACE)>
<Atom 3 (H1) of chain 0 residue 0 (ACE)>
<Atom 4 (H2) of chain 0 residue 0 (ACE)>
<Atom 5 (H3) of chain 0 residue 0 (ACE)>
<Atom 6 (N) of chain 0 residue 1 (LYS)>
<Atom 7 (CA) of chain 0 residue 1 (LYS)>
<Atom 8 (C) of chain 0 residue 1 (LYS)>
<Atom 9 (O) of chain 0 residue 1 (LYS)>
<Atom 10 (CB) of chain 0 residue 1 (LYS)>
<Atom 11 (CG) of chain 0 residue 1 (LYS)>
<Atom 12 (CD) of chain 0 residue 1 (LYS)>
<Atom 13 (CE) of chain 0 residue 1 (LYS)>
<Atom 14 (NZ) of chain 0 residue 1 (LYS)>
<Atom 15 (H2) of chain 0 residue 1 (LYS)>
<Atom 16 (HA) of chain 0 residue 1 (LYS)>
<Atom 17 (HB3) of chain 0 residue 1 (LYS)>
<Atom 18 (HB2) of chain 0 residue 1 (LYS)>
<Atom 19 (HG2) of chain 0 residue 1 (LYS)>
<Atom 20 (HG3) of chain 0 residue 1 (LYS)>
<Atom 21 (HD2) of chain 0 residue 1 (LYS)>
<Atom 22 (HD3) of chain 0 residue 1 (LYS)>
<Atom 23 (HE3) of chain 0 residue 1 (LYS)>

<Atom 3059 (H1) of chain 2 residue 630 (HOH)>
<Atom 3060 (H2) of chain 2 residue 630 (HOH)>
<Atom 3061 (O) of chain 2 residue 631 (HOH)>
<Atom 3062 (H1) of chain 2 residue 631 (HOH)>
<Atom 3063 (H2) of chain 2 residue 631 (HOH)>
<Atom 3064 (O) of chain 2 residue 632 (HOH)>
<Atom 3065 (H1) of chain 2 residue 632 (HOH)>
<Atom 3066 (H2) of chain 2 residue 632 (HOH)>
<Atom 3067 (O) of chain 2 residue 633 (HOH)>
<Atom 3068 (H1) of chain 2 residue 633 (HOH)>
<Atom 3069 (H2) of chain 2 residue 633 (HOH)>
<Atom 3070 (O) of chain 2 residue 634 (HOH)>
<Atom 3071 (H1) of chain 2 residue 634 (HOH)>
<Atom 3072 (H2) of chain 2 residue 634 (HOH)>
<Atom 3073 (O) of chain 2 residue 635 (HOH)>
<Atom 3074 (H1) of chain 2 residue 635 (HOH)>
<Atom 3075 (H2) of chain 2 residue 635 (HOH)>
<Atom 3076 (O) of chain 2 residue 636 (HOH)>
<Atom 3077 (H1) of chain 2 residue 636 (HOH)>
<Atom 3078 (H2) of chain 2 residue 636 (HOH)>
<Atom 3079 (O) of chain 2 residue 637 (HOH)>
<Atom 3080 (H1) of chain 2 residue 637 (H

<Atom 6559 (O) of chain 2 residue 1797 (HOH)>
<Atom 6560 (H1) of chain 2 residue 1797 (HOH)>
<Atom 6561 (H2) of chain 2 residue 1797 (HOH)>
<Atom 6562 (O) of chain 2 residue 1798 (HOH)>
<Atom 6563 (H1) of chain 2 residue 1798 (HOH)>
<Atom 6564 (H2) of chain 2 residue 1798 (HOH)>
<Atom 6565 (O) of chain 2 residue 1799 (HOH)>
<Atom 6566 (H1) of chain 2 residue 1799 (HOH)>
<Atom 6567 (H2) of chain 2 residue 1799 (HOH)>
<Atom 6568 (O) of chain 2 residue 1800 (HOH)>
<Atom 6569 (H1) of chain 2 residue 1800 (HOH)>
<Atom 6570 (H2) of chain 2 residue 1800 (HOH)>
<Atom 6571 (O) of chain 2 residue 1801 (HOH)>
<Atom 6572 (H1) of chain 2 residue 1801 (HOH)>
<Atom 6573 (H2) of chain 2 residue 1801 (HOH)>
<Atom 6574 (O) of chain 2 residue 1802 (HOH)>
<Atom 6575 (H1) of chain 2 residue 1802 (HOH)>
<Atom 6576 (H2) of chain 2 residue 1802 (HOH)>
<Atom 6577 (O) of chain 2 residue 1803 (HOH)>
<Atom 6578 (H1) of chain 2 residue 1803 (HOH)>
<Atom 6579 (H2) of chain 2 residue 1803 (HOH)>
<Atom 6580 (O) of ch

<Atom 10058 (H1) of chain 2 residue 2963 (HOH)>
<Atom 10059 (H2) of chain 2 residue 2963 (HOH)>
<Atom 10060 (O) of chain 2 residue 2964 (HOH)>
<Atom 10061 (H1) of chain 2 residue 2964 (HOH)>
<Atom 10062 (H2) of chain 2 residue 2964 (HOH)>
<Atom 10063 (O) of chain 2 residue 2965 (HOH)>
<Atom 10064 (H1) of chain 2 residue 2965 (HOH)>
<Atom 10065 (H2) of chain 2 residue 2965 (HOH)>
<Atom 10066 (O) of chain 2 residue 2966 (HOH)>
<Atom 10067 (H1) of chain 2 residue 2966 (HOH)>
<Atom 10068 (H2) of chain 2 residue 2966 (HOH)>
<Atom 10069 (O) of chain 2 residue 2967 (HOH)>
<Atom 10070 (H1) of chain 2 residue 2967 (HOH)>
<Atom 10071 (H2) of chain 2 residue 2967 (HOH)>
<Atom 10072 (O) of chain 2 residue 2968 (HOH)>
<Atom 10073 (H1) of chain 2 residue 2968 (HOH)>
<Atom 10074 (H2) of chain 2 residue 2968 (HOH)>
<Atom 10075 (O) of chain 2 residue 2969 (HOH)>
<Atom 10076 (H1) of chain 2 residue 2969 (HOH)>
<Atom 10077 (H2) of chain 2 residue 2969 (HOH)>
<Atom 10078 (O) of chain 2 residue 2970 (HOH)>

<Atom 14558 (H1) of chain 2 residue 4463 (HOH)>
<Atom 14559 (H2) of chain 2 residue 4463 (HOH)>
<Atom 14560 (O) of chain 2 residue 4464 (HOH)>
<Atom 14561 (H1) of chain 2 residue 4464 (HOH)>
<Atom 14562 (H2) of chain 2 residue 4464 (HOH)>
<Atom 14563 (O) of chain 2 residue 4465 (HOH)>
<Atom 14564 (H1) of chain 2 residue 4465 (HOH)>
<Atom 14565 (H2) of chain 2 residue 4465 (HOH)>
<Atom 14566 (O) of chain 2 residue 4466 (HOH)>
<Atom 14567 (H1) of chain 2 residue 4466 (HOH)>
<Atom 14568 (H2) of chain 2 residue 4466 (HOH)>
<Atom 14569 (O) of chain 2 residue 4467 (HOH)>
<Atom 14570 (H1) of chain 2 residue 4467 (HOH)>
<Atom 14571 (H2) of chain 2 residue 4467 (HOH)>
<Atom 14572 (O) of chain 2 residue 4468 (HOH)>
<Atom 14573 (H1) of chain 2 residue 4468 (HOH)>
<Atom 14574 (H2) of chain 2 residue 4468 (HOH)>
<Atom 14575 (O) of chain 2 residue 4469 (HOH)>
<Atom 14576 (H1) of chain 2 residue 4469 (HOH)>
<Atom 14577 (H2) of chain 2 residue 4469 (HOH)>
<Atom 14578 (O) of chain 2 residue 4470 (HOH)>

## Check 12/43.4

In [1]:
import pickle
import os
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTStateV2
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
# cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import argparse
import copy
from perses.dispersed import feptasks
import mdtraj as md
import numpy as np
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from openmmtools.integrators import LangevinIntegrator
import time
from tqdm import tqdm

INFO:numexpr.utils:Note: NumExpr detected 48 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


conducting subsequent work with the following platform: CUDA


INFO:rdkit:Enabling RDKit 2020.09.1 jupyter extensions


In [2]:
# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)


In [37]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/12/43/"
resid = '38'
old_aa_name = 'PHE'
new_aa_name = 'TRP'
sim_number = 4
length = 1
move_length = 3
timestep = 4
radius = 0.2

if sim_number == 1:
    phase = 'apo'
    name = old_aa_name
    state = 0
elif sim_number == 2:
    phase = 'apo'
    name = new_aa_name
    state = 1
elif sim_number == 3:
    phase = 'complex'
    name = old_aa_name
    state = 0
elif sim_number == 4:
    phase = 'complex'
    name = new_aa_name
    state = 1



In [38]:
i = os.path.basename(os.path.dirname(outdir))
path = os.path.join(outdir, f"{i}_{phase}_{state}.pickle")
htf = pickle.load(open(path, "rb" ))


In [39]:
# Build REST factory
_logger.info("Generating REST factory")
_logger.info(f"radius:{radius} nm")
query_indices = [atom.index for atom in list(htf.hybrid_topology.residues)[int(resid)].atoms]
_logger.info(f"query indices {query_indices}")
traj = md.Trajectory(np.array(htf.hybrid_positions), htf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=solute_atoms)[0])
factory = RESTTopologyFactory(htf.hybrid_system, solute_region=rest_atoms)

_logger.info("Generating REST states")
# Get REST system
REST_system = factory.REST_system

# Create states for each replica
n_replicas = 12  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 1200 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

# Create reference thermodynamic state
lambda_zero_alchemical_state = RESTStateV2.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

context_cache = cache.ContextCache()

# Create thermodynamics states
sampler_state =  SamplerState(htf.hybrid_positions, box_vectors=htf.hybrid_system.getDefaultPeriodicBoxVectors())
beta_0 = 1/(kB*T_min)
thermodynamic_state_list = []
sampler_state_list = []
for temperature in temperatures:
    beta_m = 1/(kB*temperature)
    compound_thermodynamic_state_copy = copy.deepcopy(compound_thermodynamic_state)
    compound_thermodynamic_state_copy.set_alchemical_parameters(beta_0, beta_m)
    thermodynamic_state_list.append(compound_thermodynamic_state_copy)

    # now generating a sampler_state for each thermodynamic state, with relaxed positions
    # context, context_integrator = context_cache.get_context(compound_thermodynamic_state_copy)
    feptasks.minimize(compound_thermodynamic_state_copy, sampler_state)
    sampler_state_list.append(copy.deepcopy(sampler_state))

INFO:root:Generating REST factory
INFO:root:radius:0.2 nm
INFO:root:query indices [604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 29473, 29474, 29475, 29476, 29477, 29478, 29479, 29480, 29481, 29482, 29483, 29484, 29485, 29486, 29487, 29488, 29489, 29490]
INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=6.7997, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=6.7997, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=6.7997), unit=nanometer)]
INFO:REST:No unknown forces.
INFO:root:Generating REST states


In [40]:
# Define simulation parameters
temperature = 298 * unit.kelvin
nsteps = 250 # 1 ps
timestep = 2.0 * unit.femtosecond
platform_name = 'CUDA'
collision_rate = 1.0 / unit.picoseconds

In [41]:
# Set up integrator
integrator = LangevinIntegrator(temperature, collision_rate, timestep)

# Set up context
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')

context = thermodynamic_state_list[0].create_context(integrator, platform=platform)    

context.setPeriodicBoxVectors(*htf.hybrid_system.getDefaultPeriodicBoxVectors())
context.setPositions(htf.hybrid_positions)
context.setVelocitiesToTemperature(temperature)
print(compound_thermodynamic_state.reduced_potential(context))

# Minimize
openmm.LocalEnergyMinimizer.minimize(context)



235383911.27275702


In [42]:
# Run equilibration
times = []
pos_new = []
pos_old = []
for step in tqdm(range(nsteps)):
    initial_time = time.time()
    integrator.step(1)
    elapsed_time = (time.time() - initial_time) * unit.seconds
    pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
    old_pos = htf.old_positions(pos)
    new_pos = htf.new_positions(pos)
    pos_new.append(new_pos)
    pos_old.append(old_pos)
    print(f'took: {elapsed_time / unit.seconds} seconds')
    times.append(elapsed_time / unit.seconds)


  0%|          | 1/250 [00:00<03:06,  1.33it/s]

took: 0.0022559165954589844 seconds


  1%|          | 2/250 [00:01<03:10,  1.30it/s]

took: 0.0005383491516113281 seconds


  1%|          | 3/250 [00:02<03:07,  1.31it/s]

...

took: 0.0005452632904052734 seconds


100%|█████████▉| 249/250 [03:10<00:00,  1.27it/s]

took: 0.0008349418640136719 seconds


100%|██████████| 250/250 [03:11<00:00,  1.31it/s]

took: 0.003756284713745117 seconds


In [30]:
len(pos_new)

21

In [28]:
pos_new[0].shape[0]

29477

In [29]:
pos_new[0].shape[1]

3

In [31]:
pos_new_arr = np.zeros(shape=(len(pos_new), pos_new[0].shape[0], pos_new[0].shape[1]))
for i, pos in enumerate(pos_new):
    pos_new_arr[i] = pos.value_in_unit_system(unit.md_unit_system)

In [32]:
pos_old_arr = np.zeros(shape=(len(pos_old), pos_old[0].shape[0], pos_old[0].shape[1]))
for i, pos in enumerate(pos_old):
    pos_old_arr[i] = pos.value_in_unit_system(unit.md_unit_system)

In [33]:
traj_new = md.Trajectory(pos_new_arr, md.Topology.from_openmm(htf._topology_proposal.new_topology))
traj_new.save(outdir + f"new.dcd")

In [35]:
traj_new[0].save(outdir + f"new.pdb")

In [36]:
traj_old = md.Trajectory(pos_old_arr, md.Topology.from_openmm(htf._topology_proposal.old_topology))
traj_old.save(outdir + f"old.dcd")
traj_old[0].save(outdir + f"old.pdb")